In [1]:
# Import classes
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import LlamaCpp
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.document_loaders import PyPDFDirectoryLoader
import warnings
warnings.filterwarnings("ignore")

In [6]:
# Create a PyPDFDirectoryLoader object with the directory path "Data/"
loader = PyPDFDirectoryLoader("Data/")

# Load data using the PyPDFDirectoryLoader object
data = loader.load()

In [7]:
# Create a RecursiveCharacterTextSplitter object with max size of each chunk and overlap between chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10000, chunk_overlap=20)

# Split documents into chunks using the RecursiveCharacterTextSplitter object
text_chunks = text_splitter.split_documents(data)

In [8]:
# Get the number of text chunks generated by the splitting process
len(text_chunks)

459

In [9]:
# Access the first text chunk generated by the splitting process
text_chunks[0]

Document(metadata={'source': 'Data\\The-AI-Act-Final.pdf', 'page': 0}, page_content='European Parliament\n2019-2024\nTEXTS ADOPTED\nP9_TA(2024)0138\nArtificial Intelligence Act\nEuropean Parliament legislative resolution of 13 March 2024 on the proposal for a \nregulation of the European Parliament and of the Council on laying down harmonised \nrules on Artificial Intelligence (Artificial Intelligence Act) and amending certain Union \nLegislative Acts (COM(2021)0206 – C9-0146/2021 – 2021/0106(COD))\n(Ordinary legislative procedure: first reading)\nThe European Parliament,\n– having regard to the Commission proposal to Parliament and the Council \n(COM(2021)0206),\n– having regard to Article 294(2) and Articles 16 and 114 of the Treaty on the \nFunctioning of the European Union, pursuant to which the Commission submitted the \nproposal to Parliament (C9-0146/2021),\n– having regard to Article 294(3) of the Treaty on the Functioning of the European Union,\n– having regard to the opinion 

In [10]:
# Initialize HuggingFaceEmbeddings with the specified model_name
# This will load the pre-trained model "all-MiniLM-L6-v2" for generating embeddings
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

In [11]:
# Create a FAISS vector store from the text chunks using the specified embeddings
# This will generate vectors for each text chunk based on the provided embeddings
vector_store = FAISS.from_documents(text_chunks, embedding=embeddings)

In [12]:
#   Initialize LlamaCpp with specified parameters:
#   - streaming: Enables streaming mode for continuous input processing.
#   - model_path: Path to the LLM model file "mistral-7b-instruct-v0.1.Q4_K_M.gguf".
#   - temperature: Softmax temperature for controlling generation randomness.
#   - top_p: Top-p (nucleus) sampling threshold for controlling generation diversity.
#   - verbose: Enables verbose mode for detailed logging.
#   - n_ctx: Size of the input context window for the model.
llm = LlamaCpp(
    streaming = True,
    model_path="mistral-7b-instruct-v0.1.Q4_K_M.gguf",
    temperature=0.85,
    top_p=1,
    verbose=True,
    n_ctx=4096
)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from mistral-7b-instruct-v0.1.Q4_K_M.gguf (version GGUF V2)
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistralai_mistral-7b-instruct-v0.1
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.atte

In [13]:
# Create a RetrievalQA object using the specified parameters:
#   - llm: LlamaCpp instance used for language model inference.
#   - chain_type: Type of QA chain to create ("stuff" in this case).
#   - retriever: Vector store converted to a retriever with search arguments {"k": 2}.
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=vector_store.as_retriever(search_kwargs={"k": 2})
)

In [14]:
# Define a query for the QA system to process
query_01 = "What is the purpose of the AI Act?"

In [15]:
# Execute the QA system on the provided query to retrieve an answer
qa.run(query_01)

llama_perf_context_print:        load time =   26721.10 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   515 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /    89 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   37036.27 ms /   604 tokens


' The purpose of the AI Act is to improve the functioning of the internal market and promote the uptake of human-centric and trustworthy artificial intelligence (AI), while ensuring a high level of protection of health, safety, fundamental rights enshrined in the Charter of Fundamental Rights, including democracy, the rule of law and environmental protection, against the harmful effects of AI systems in the Union, and to support innovation.'

In [16]:
# Define a query for the QA system to process
query_02 = "I want to share AI-generated content, does it have to be tagged somehow? Can you give me the original article in the AI Act that you have used as a source?" 

In [17]:
# Execute the QA system on the provided query to retrieve an answer
qa.run(query_02)

Llama.generate: 46 prefix-match hit, remaining 649 prompt tokens to eval
llama_perf_context_print:        load time =   26721.10 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   649 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   154 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   50942.02 ms /   803 tokens


" No, AI-generated content doesn't need to be tagged. According to the AI Act, providers of general-purpose AI models do not have to obtain an authorization from rightsholders if they want to carry out text and data mining over copyrighted works, unless the rights to opt out has been expressly reserved in an appropriate manner. However, providers of these models must comply with Union copyright law, and identify and comply with a reservation of rights expressed pursuant to Article 4(3) of Directive (EU) 2019/790. It is not specified whether there is any requirement for the content used to train the general-purpose AI model to be tagged in this context."

In [18]:
# Define a query for the QA system to process
query_03 = "Can you give me some examples of unacceptable AI practices according to the AI Act?"

In [19]:
# Execute the QA system on the provided query to retrieve an answer
qa.run(query_03)

Llama.generate: 45 prefix-match hit, remaining 523 prompt tokens to eval
llama_perf_context_print:        load time =   26721.10 ms
llama_perf_context_print: prompt eval time =       0.00 ms /   523 tokens (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:        eval time =       0.00 ms /   166 runs   (    0.00 ms per token,      inf tokens per second)
llama_perf_context_print:       total time =   44031.72 ms /   689 tokens


" Yes, some examples of unacceptable AI practices according to the AI Act are:\n\n* Using subliminal techniques beyond a person's consciousness or purposefully manipulative or deceptive techniques with the objective, or effect, of materially distorting the behavior of a person or group of persons by appreciably impairing their ability to make an informed decision.\n* Placing on the market, putting into service, or using an AI system that deploys such techniques, unless it has been assessed as not high-risk and is subject to registration with national authorities.\n* The use of AI systems that pose a significant risk of harm to natural persons' health, safety, or fundamental rights, without a proper assessment and documentation of the risk, or without being registered with national authorities."